In [0]:
!git clone "https://github.com/udacity/deep-learning.git"

Cloning into 'deep-learning'...
remote: Enumerating objects: 1450, done.
remote: Total 1450 (delta 0), reused 0 (delta 0), pack-reused 1450
Receiving objects: 100% (1450/1450), 49.28 MiB | 28.90 MiB/s, done.
Resolving deltas: 100% (591/591), done.


In [0]:
%cd deep-learning/tv-script-generation

/content/deep-learning/tv-script-generation


In [0]:
%load helper.py

In [0]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

In [0]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




In [0]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    
    word_counts = Counter(text)                  
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    
    int_to_vocab = {i:word for i,word in enumerate(sorted_vocab)}
    vocab_to_int = {word:i for i,word in int_to_vocab.items()}
    
    return (vocab_to_int, int_to_vocab)
  
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    
    punct_to_token = {'.':'Period', ',':'Comma', '"':'Quotation_mark', ';':'Semicolon', '!':'Exclamation_mark', '?':'Question_mark', '(':'Left_Parentheses', ')':'Right_Parentheses', '--':'Dash', '\n':'Return'}
    
    return punct_to_token

tests.test_tokenize(token_lookup)

Tests Passed


In [0]:
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [0]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()


In [0]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.13.1
Default GPU Device: /device:GPU:0


In [0]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    
    inputs = tf.placeholder(tf.int32, shape=[None,None], name='input')
    targets = tf.placeholder(tf.int32, shape=[None,None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return (inputs, targets, learning_rate)
  
tests.test_get_inputs(get_inputs)


Tests Passed


In [0]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, astor, protobuf, tensorboard, tensorflow-estimator, keras-applications, wheel, gast, numpy, grpcio, keras-preprocessing, termcolor, absl-py
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])

    
    initial = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial, name='initial_state')
    
    return (cell, initial_state)
  
tests.test_get_init_cell(get_init_cell)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Tests Passed


In [0]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embedded =  tf.nn.embedding_lookup(embedding, input_data)                        
    
    return embedded
  
tests.test_get_embed(get_embed)

Instructions for updating:
Colocations handled automatically by placer.
Tests Passed


In [0]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    
    outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs, dtype=tf.float32)
    final_state = tf.identity(state, name='final_state')
    
    return (outputs, final_state)

tests.test_build_rnn(build_rnn)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Tests Passed


In [0]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    embedded = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return (logits, final_state)
  
tests.test_build_nn(build_nn)  

Tests Passed


In [0]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    
    count=0
    n_batches = int(len(int_text)//(batch_size*seq_length))
    batch_i = np.zeros((n_batches, batch_size, seq_length))
    batch_t = np.zeros((n_batches, batch_size, seq_length))
    batch_array = np.zeros((2)
    batch = np.zeros((n_batches,2))
    
    for i in range(batch_size):
      for n in range(n_batches):
        for  j in range(seq_length):
          
          batch_i[n][i][j] = int_text[count]
          if(count==(n*i*j)-1):
            batch_t[n][i][j] = int_text[0]
          batch_t[n][i][j] = int_text[count+1]
          count += 1
          
    for n in range(n_batches):
      for i in range(batch_size):
        for  j in range(seq_length):
          
          batch[n][0] = batch_i[n][i][j]
          batch[n][1] = batch_t[n][i][j]
          batch_array[0] = 
      
    
    
    return batch_array
  
tests.test_get_batches(get_batches)

SyntaxError: ignored

In [0]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    characters_per_batch = batch_size * seq_length #640
    num_batches = len(int_text)//(characters_per_batch) #7 

    # clip arrays to ensure we have complete batches for inputs, targets same but moved one unit over,
    # last character in target array is first of input array 
    # both have shape (4480, )
    input_data = np.array(int_text[ : (num_batches * characters_per_batch)])
    target_data = np.array(int_text[1 : (num_batches * characters_per_batch)] + [int_text[0]])

    inputs = input_data.reshape(batch_size, -1)
    targets = target_data.reshape(batch_size, -1)
    # both now have shape (7,640)
    
    inputs = np.split(inputs, num_batches, 1) #num_batches is 7
    targets = np.split(targets, num_batches, 1)
    
    batches = np.array(list(zip(inputs, targets)))
    batches = batches.reshape(num_batches, 2, batch_size, seq_length)

    return batches
  

tests.test_get_batches(get_batches)

Tests Passed


In [0]:
# Number of Epochs
num_epochs = 20
# Batch Size
batch_size = 5
# RNN Size
rnn_size = 3
# Embedding Dimension Size
embed_dim = 6
# Sequence Length
seq_length = 2
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 4

save_dir = './save'

In [0]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)


Instructions for updating:
Use tf.cast instead.


In [0]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/6910   train_loss = 8.822
Epoch   0 Batch    4/6910   train_loss = 8.806
Epoch   0 Batch    8/6910   train_loss = 8.771
Epoch   0 Batch   12/6910   train_loss = 8.780
Epoch   0 Batch   16/6910   train_loss = 8.683
Epoch   0 Batch   20/6910   train_loss = 8.744
Epoch   0 Batch   24/6910   train_loss = 8.560
Epoch   0 Batch   28/6910   train_loss = 8.673
Epoch   0 Batch   32/6910   train_loss = 8.310
Epoch   0 Batch   36/6910   train_loss = 8.474
Epoch   0 Batch   40/6910   train_loss = 8.450
Epoch   0 Batch   44/6910   train_loss = 8.408
Epoch   0 Batch   48/6910   train_loss = 8.112
Epoch   0 Batch   52/6910   train_loss = 7.992
Epoch   0 Batch   56/6910   train_loss = 8.274
Epoch   0 Batch   60/6910   train_loss = 8.182
Epoch   0 Batch   64/6910   train_loss = 7.585
Epoch   0 Batch   68/6910   train_loss = 7.774
Epoch   0 Batch   72/6910   train_loss = 7.578
Epoch   0 Batch   76/6910   train_loss = 7.152
Epoch   0 Batch   80/6910   train_loss = 7.503
Epoch   0 Bat

In [0]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

In [0]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

In [0]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return (input_tensor, initial_state_tensor, final_state_tensor, probs_tensor )
    
  
tests.test_get_tensors(get_tensors)  

Tests Passed


In [0]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    
    cumulative_sum = np.cumsum(probabilities) # array with 4 values, sums items before
    chooser = np.sum(probabilities) * np.random.rand(1) # generates an array of possible values, uniform distribution
    word = int_to_vocab[int(np.searchsorted(cumulative_sum, chooser))] #finds index of word
    
    return word
  
tests.test_pick_word(pick_word)  

Tests Passed


In [0]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
        
        # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(drinking from drunk with my wrong our million in with some burps) ya would they heard my best goodnight sure room) edna just too motor you know by my kids.
moe_szyslak:.
moe_szyslak: i buy that.
lisa_simpson: it's yeah, get an beginning.
homer_simpson: now
homer_simpson:(quietly! he's phone will school. who lucky a blaze, we need a day of that and(making brainheaded."(to be town's while.
moe_szyslak: hey". one easy through the car, well. was real guys at the gotta drink, hear the couple ways a parking friend need did.
inclination, ah, we've be with him in make neat's-foot!
moe_szyslak: well, i'm grunts with my secret buy insightful.
homer_simpson:(the bottom gonna car see her paid moans) homer_simpson: it's come, but.
barney_gumble: while
moe_szyslak: i rubbed that he's homer caught?
moe_szyslak:(a morlocks make me as a cash together!
do, 